In [130]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from unidecode import unidecode

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

Selenium Setup

In [183]:
mode= 'women'
url = f"https://www.sports-reference.com/cbb/postseason/{mode}/2023-ncaa.html"

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(0.5)
driver.maximize_window()
driver.get(url)
wait = WebDriverWait(driver, 20)

actions = ActionChains(driver)

/var/folders/ds/y506zm_56zv14d0zsmmsnz_c0000gn/T/ipykernel_80920/2871711139.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Scrape sportsref bracket to create df

In [188]:
# keep track of teams already traversed
schools = []
urls = []
rounds = []
brackets = driver.find_elements(By.XPATH,'//*[@id="brackets"]/div')

# print(len(brackets))

# loop through all 4 conferences + final four
# start at final four and go backwards
for index, conf in enumerate(reversed(brackets)):

    round_elems = conf.find_elements(By.XPATH, './div/div')
    
    # loop through each section of the bracket (1,2,4,...)
    for r_index, r in enumerate(reversed(round_elems)):
        teams = r.find_elements(By.XPATH, './div/div/a[1]')
        for t in teams:
            team_name = t.get_attribute('textContent')
            if team_name and team_name not in schools:
                schools.append(team_name)
                urls.append(t.get_attribute('href'))
                if(index == 0):
                    rounds.append(pow(2, r_index))
                else:
                    rounds.append(pow(2, r_index+2))

df = pd.DataFrame({'school': schools, 'round':rounds,'sportsref_url':urls})
print(df.to_string())

                      school  round                                                                        sportsref_url
0            San Diego State      2           https://www.sports-reference.com/cbb/schools/san-diego-state/men/2023.html
1                      UConn      2               https://www.sports-reference.com/cbb/schools/connecticut/men/2023.html
2           Florida Atlantic      4          https://www.sports-reference.com/cbb/schools/florida-atlantic/men/2023.html
3                 Miami (FL)      4                  https://www.sports-reference.com/cbb/schools/miami-fl/men/2023.html
4                    Gonzaga      8                   https://www.sports-reference.com/cbb/schools/gonzaga/men/2023.html
5                   Arkansas     16                  https://www.sports-reference.com/cbb/schools/arkansas/men/2023.html
6                       UCLA     16                      https://www.sports-reference.com/cbb/schools/ucla/men/2023.html
7                     Kansas    

Scrape team sportsref pages to get team stats and some player stats

In [189]:
min_return=[]
scoring_return=[]
player_data=[]
for index, row in df.iterrows():
    team_page = rq.get(row['sportsref_url'])
    if(team_page.status_code != 200):
        print(f'request failed with status code {team_page.status_code}, url: {row["sportsref_url"]}')
        break

    soup = bs(team_page.content)
    return_fact = soup.find('div', id='tfooter_roster')
    # print(return_fact)
    if(return_fact is not None):
        return_fact = return_fact.small.text.split()
        min_return.append(return_fact[0])
        scoring_return.append(return_fact[5])
    else:
        min_return.append(None)
        scoring_return.append(None)

    p_game_table = soup.find('table',id='per_game')
    total_table = soup.find('table',id='totals').find('tbody')

    p_game_body = p_game_table.find('tbody')
    p_game_rows = p_game_body.find_all('tr')
    for t_row in p_game_rows:
        cells = t_row.find_all('td')
        name = cells[0].a.text
        avg_min = cells[3].text
        avg_pts = t_row.find('td',attrs={'data-stat':'pts_per_g'}).text
        sportsref_url = f"https://www.sports-reference.com{unidecode(cells[0].a['href'])}"

        total_min = total_table.find('td',string=name).find_next_sibling('td',attrs={'data-stat':'mp'}).text
        total_pts= total_table.find('td',string=name).find_next_sibling('td',attrs={'data-stat':'pts'}).text

        player_data.append([name, sportsref_url, row['school'], avg_min, avg_pts, total_min, total_pts])

    time.sleep(3)
    # break 

# print(player_data)
df['percent_minutes_return_21_22'] = min_return
df['percent_scoring_return_21_22'] = scoring_return


player df
- name
- team
- transfer? look at one year transfers for now
- avg mins per game 22-23
- total mins 22-23
- avg pts per game 22-23
- total pts 22-23

In [192]:
# determine if a player transferred or not by looking at what school they played at in 21-22 vs 22-23
# TODO: transfer conference col

player_df = pd.DataFrame(player_data, columns=['name','sportsref_url','school','avg_min','avg_pts','total_min','total_pts'])
transfer_col=[]
for index,row in player_df.iterrows():
    page = rq.get(row['sportsref_url'])
    if(page.status_code != 200):
        transfer_col.append(False)
        print(f'request failed with status code {page.status_code}, url: {row["sportsref_url"]}')
        continue

    soup = bs(page.content)
    # print(row['name'])
    per_game_table = soup.find('table',id='players_per_game')
    school_22_23 = per_game_table.find('a',string="2022-23").find_next('td',attrs={'data-stat':'school_name'}).text
    school_21_22 = per_game_table.find('a',string="2021-22")
    
    # first year
    if(school_21_22 is None):
        # print('not transfer')
        transfer_col.append(False)
        time.sleep(3)
        continue
    
    school_21_22 = school_21_22.find_next('td',attrs={'data-stat':'school_name'}).text

    if(school_21_22 != school_22_23):
        # print('transferred')
        transfer_col.append(True)
    else:
        # print('not transfer')
        transfer_col.append(False)

    time.sleep(2)

player_df['transfer']=transfer_col

In [193]:
player_df['avg_min'] = pd.to_numeric(player_df['avg_min'])
player_df['avg_pts'] = pd.to_numeric(player_df['avg_pts'])
player_df['total_min'] = pd.to_numeric(player_df['total_min'])
player_df['total_pts'] = pd.to_numeric(player_df['total_pts'])

player_df.to_csv(f'{mode}/{mode}_player_df.csv')

In [ ]:
avg_transfer_mins_p_game=[]
avg_transfer_pts_p_game=[]
avg_transfer_mins_total=[]
avg_transfer_pts_total=[]
num_transfers_list=[]
# loop through each school
for index,row in df.iterrows():
    school = row['school']

    # average transfer stats
    num_transfers = 0
    avg_t_min = 0 # average minutes played by transfers per game
    avg_t_pts = 0 # avg points scored by transfers per game
    avg_t_min_total = 0 # average total minutes played by transfers
    avg_t_pt_total = 0 # average total points scored by transfers
    players = player_df[player_df['school'] == school]
    for index, row in players.iterrows():
        if row['transfer']:
            num_transfers+=1
            avg_t_min += row['avg_min']
            avg_t_pts += row['avg_pts']
            avg_t_min_total += row['total_min']
            avg_t_pt_total += row['total_pts']
            
    if(num_transfers != 0):
        avg_t_min /= num_transfers
        avg_t_pts /= num_transfers
        avg_t_min_total /= num_transfers
        avg_t_pt_total /= num_transfers
    # print(num_transfers)
    num_transfers_list.append(num_transfers)
    avg_transfer_mins_p_game.append(avg_t_min)
    avg_transfer_pts_p_game.append(avg_t_pts)
    avg_transfer_mins_total.append(avg_t_min_total)
    avg_transfer_pts_total.append(avg_t_pt_total)
df['num_transfers'] = num_transfers_list
df['avg_transfer_mins_p_game'] = avg_transfer_mins_p_game
df['avg_transfer_pts_p_game'] = avg_transfer_pts_p_game
df['avg_transfer_mins_total'] = avg_transfer_mins_total
df['avg_transfer_pts_total'] = avg_transfer_pts_total

In [195]:
df['percent_minutes_return_21_22']=df['percent_minutes_return_21_22'].apply(lambda x: x.replace('%',''))
df['percent_scoring_return_21_22']=df['percent_scoring_return_21_22'].apply(lambda x: x.replace('%',''))
df['percent_minutes_return_21_22'] = pd.to_numeric(df['percent_minutes_return_21_22'])
df['percent_scoring_return_21_22'] = pd.to_numeric(df['percent_scoring_return_21_22'])
df.to_csv(f'{mode}/{mode}_team_df_w_transfer_stats.csv')
df

,school,round,sportsref_url,percent_minutes_return_21_22,percent_scoring_return_21_22,num_transfers,avg_transfer_mins_p_game,avg_transfer_pts_p_game,avg_transfer_mins_total,avg_transfer_pts_total
0,San Diego State,2,https://www.sports-reference.com/cbb/schools/s...,67.2,68.9,2,24.350,8.600000,935.500000,331.500000
1,UConn,2,https://www.sports-reference.com/cbb/schools/c...,34.4,34.0,4,18.475,5.800000,707.250000,222.000000
2,Florida Atlantic,4,https://www.sports-reference.com/cbb/schools/f...,77.5,82.1,2,14.000,3.050000,496.500000,104.500000
3,Miami (FL),4,https://www.sports-reference.com/cbb/schools/m...,49.8,46.5,2,30.200,13.350000,1086.500000,481.500000
4,Gonzaga,8,https://www.sports-reference.com/cbb/schools/g...,68.3,68.7,2,12.700,5.400000,442.500000,187.000000
...,...,...,...,...,...,...,...,...,...,...
59,Louisiana,64,https://www.sports-reference.com/cbb/schools/l...,64.6,70.5,1,23.100,10.900000,784.000000,370.000000
60,Providence,64,https://www.sports-reference.com/cbb/schools/p...,25.5,26.9,5,23.500,9.400000,768.000000,308.000000
61,Montana State,64,https://www.sports-reference.com/cbb/schools/m...,52.2,55.2,3,24.900,6.700000,853.666667,230.333333
62,USC,64,https://www.sports-reference.com/cbb/schools/s...,47.0,46.6,0,0.000,0.000000,0.000000,0.000000
